In [ ]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
import math
import hanja
import re
import string
import operator
import random
import matplotlib.pyplot as plt
import itertools
import cnouns as cn
import check_utils as cu
import deep_utils as du
from sklearn.metrics import adjusted_rand_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score
from time import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from datetime import datetime
from sklearn.decomposition import PCA
from gensim import models
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
import gensim.models.doc2vec
from collections import OrderedDict
from gensim.models.doc2vec import LabeledSentence

import multiprocessing
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
import cPickle as pickle
from spherecluster import SphericalKMeans
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

from random import shuffle

from collections import namedtuple

Articles = namedtuple('Articles', 'words tags split')

# Load Data

In [ ]:
stop = 0
# test = 1
test = 2

In [ ]:
train_df = pd.read_pickle("../datastore/deep_df.p")

models_by_name = OrderedDict()
models_by_name['Doc2Vec(dm/c,d100,n5,w5,mc2,t8)'] = Doc2Vec.load("../datastore/d2v-dmc_%d.p" % test)
models_by_name['Doc2Vec(dbow,d100,n5,mc2,t8)'] = Doc2Vec.load("../datastore/d2v-dbow_%d.p" % test)
models_by_name['Doc2Vec(dm/m,d100,n5,w10,mc2,t8)'] = Doc2Vec.load("../datastore/d2v-dmm_%d.p" % test)

In [ ]:
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([models_by_name['Doc2Vec(dbow,d100,n5,mc2,t8)'], models_by_name['Doc2Vec(dm/m,d100,n5,w10,mc2,t8)']])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([models_by_name['Doc2Vec(dbow,d100,n5,mc2,t8)'], models_by_name['Doc2Vec(dm/c,d100,n5,w5,mc2,t8)']])

# Select Model

In [ ]:
# dm = models_by_name['dbow+dmm']
# dm = models_by_name['dbow+dmc']
dm = models_by_name['Doc2Vec(dm/c,d100,n5,w5,mc2,t8)']

In [ ]:
doc_arr = dm.docvecs
inp = np.array(doc_arr)

# Clustering

In [ ]:
n_cluster = 200

model = KMeans
# model = SphericalKMeans

In [ ]:
d_km = model(n_clusters=n_cluster, n_jobs=-1)
d_km.fit(inp)

In [ ]:
clusters = d_km.labels_.tolist()
train_df['cluster'] = clusters

# Scoring

In [ ]:
print "inertia : ", d_km.inertia_
print "silhouette score : ", silhouette_score(inp, d_km.labels_)

In [ ]:
sorted_cluster = cu.sort_count(train_df, range(n_cluster))
sorted_cluster

In [ ]:
cu.test_print(32, d_km, inp, train_df)

In [ ]:
print dm.docvecs.similarity(d1=2183, d2=2165)
print dm.docvecs.similarity(d1=2267, d2=2328)

In [ ]:
cu.test_similar(1, dm.docvecs, train_df, threadsold=0.5, is_last = False)

# Stop

In [ ]:
if(stop==1):
    asdasd

# Similarity Clustering

In [ ]:
train_df = pd.read_pickle("../datastore/deep_result_df.p")
centers = pickle.load(open("../datastore/deep_centers.p", "rb"))

In [ ]:
du.calc_similarity(train_df, dm.docvecs, centers)

In [ ]:
train_df[train_df.similarity<1.0]

# Similarity Scoring

In [ ]:
score = du.similarity_iner_score(centers, train_df, dm.docvecs)

In [ ]:
size_1 = score[score.cnt==1]
countby = score[score.cnt>10]
print "total:", len(score), ", size_1:",len(size_1), ", countby:", len(countby)
ss = countby.sum(axis=0)
print "distance:", ss['distance'] * 100
print "variance:", ss['variance']
print "similarity:", (ss['similarity'] * 100)/len(countby)
print "cohesion:", ss['cohesion']

In [ ]:
score.sort_values('similarity', ascending=True)[:10]

In [ ]:
score.sort_values('cnt', ascending=False)[:10]

In [ ]:
countby.sort_values('variance', ascending=True)[:10]

In [ ]:
countby.sort_values('similarity', ascending=False)[:10]

In [ ]:
countby.sort_values('cohesion', ascending=False)[:10]

# Get Topics

In [ ]:
topics = pickle.load(open("../datastore/deep_topics.p", "rb"))

In [ ]:
for idx, row in countby.sort_values('cohesion', ascending=False)[:10].iterrows():
    print du.test_print(row.cluster, train_df, dm.docvecs, centers, topics, countby)
    print "\n------------------------------------------------------------\n"

In [ ]:
times = pickle.load(open("../datastore/deep_times.p", "rb"))

In [ ]:
elapsed_time = 0
for key, value in times.iteritems():
    elapsed_time = elapsed_time + value["elapsed"]
    print key, " ", value["elapsed"]
    
print "\nelapsed_time ", elapsed_time

# TODO

* 뉴스 유사도 정렬
* 중앙에서 가까운 뉴스 개수 활용하여 이슈 선정